In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle

In [13]:
df=pd.read_csv("IMDBcleaned.csv")
df

,name,country,year,director,cast,rating,genre,rating_count,duration,description
0,The Shawshank Redemption,United States,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton",9.3,Drama,2949951,142,A banker convicted of uxoricide forms a friend...
1,The Godfather,United States,1972,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola",9.2,"Crime, Drama",2056341,175,The aging patriarch of an organized crime dyna...
2,The Dark Knight,United States,2008,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer",9.0,"Action, Crime, Drama",2925228,152,When a menace known as the Joker wreaks havoc ...
3,The Godfather Part II,United States,1974,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo",9.0,"Crime, Drama",1389782,202,The early life and career of Vito Corleone in ...
4,12 Angry Men,United States,1957,Sidney Lumet,Reginald Rose,9.0,"Crime, Drama",887543,96,The jury in a New York City murder trial is fr...
...,...,...,...,...,...,...,...,...,...,...
245,Cool Hand Luke,United States,1967,Stuart Rosenberg,"Donn Pearce, Frank Pierson, Hal Dresner",8.1,"Crime, Drama",191299,127,A laid-back Southern man is sentenced to two y...
246,It Happened One Night,United States,1934,Frank Capra,"Clark Gable, Claudette Colbert, Walter Connolly",8.1,"Comedy, Romance",114437,105,A rogue reporter trailing a runaway heiress fo...
247,Gangs of Wasseypur,India,2012,Anurag Kashyap,"Akhilesh Jaiswal, Anurag Kashyap, Sachin K. Ladia",8.2,"Action, Comedy, Crime",105645,321,A clash between Sultan and Shahid Khan leads t...
248,Drishyam,India,2015,Nishikant Kamat,"Ajay Devgn, Shriya Saran, Tabu",8.2,"Crime, Drama, Mystery",97800,163,Desperate measures are taken by a man who trie...


In [14]:
columns=['cast','director','genre','title','description']
def get_important_features(df1):
    important_features=[]
    for i in range(df1.shape[0]):
        important_features.append(df1['name'][i]+" "+df1['director'][i]+" "+df1['genre'][i]+" "+df1['description'][i]+" "+df1['cast'])
    return important_features

CREATING COLUMN TO HOLD THE COMBINED STRINGS

In [15]:
df['important_features']=get_important_features(df)

In [17]:
df.head(5)

,name,country,year,director,cast,rating,genre,rating_count,duration,description,important_features
0,The Shawshank Redemption,United States,1994,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton",9.3,Drama,2949951,142,A banker convicted of uxoricide forms a friend...,0 The Shawshank Redemption Frank Darabont...
1,The Godfather,United States,1972,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola",9.2,"Crime, Drama",2056341,175,The aging patriarch of an organized crime dyna...,0 The Godfather Francis Ford Coppola Crim...
2,The Dark Knight,United States,2008,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Goyer",9.0,"Action, Crime, Drama",2925228,152,When a menace known as the Joker wreaks havoc ...,0 The Dark Knight Christopher Nolan Actio...
3,The Godfather Part II,United States,1974,Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo",9.0,"Crime, Drama",1389782,202,The early life and career of Vito Corleone in ...,0 The Godfather Part II Francis Ford Copp...
4,12 Angry Men,United States,1957,Sidney Lumet,Reginald Rose,9.0,"Crime, Drama",887543,96,The jury in a New York City murder trial is fr...,"0 12 Angry Men Sidney Lumet Crime, Drama ..."


In [19]:
df['important_features'] = df['important_features'].fillna('').astype('str')

# Initialize TfidfVectorizer with stop words removal
tfidf = TfidfVectorizer(stop_words='english')

# Apply TfidfVectorizer on 'important_features'
tfidf_matrix = tfidf.fit_transform(df['important_features'])

# Check the shape of the tfidf matrix (rows, terms)
tfidf_matrix.shape

(250, 834)

In [26]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df.index, index=df['name']).drop_duplicates()
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=df['name'].iloc[movie_indices]
    dict={"Movies":movies}
    final_df=pd.DataFrame(dict)
    final_df = final_df.reset_index(drop=True)
    return final_df

In [28]:
get_recommendations('The Shawshank Redemption')

,Movies
0,The Green Mile
1,It&apos;s a Wonderful Life
2,It Happened One Night
3,Mr. Smith Goes to Washington
4,Oppenheimer


In [29]:
df.info()
new = df.drop(columns=['year','duration','genre','rating','director','cast','rating_count','description'])

pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(cosine_sim,open('similarity.pkl','wb'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                250 non-null    object 
 1   country             250 non-null    object 
 2   year                250 non-null    int64  
 3   director            250 non-null    object 
 4   cast                250 non-null    object 
 5   rating              250 non-null    float64
 6   genre               250 non-null    object 
 7   rating_count        250 non-null    int64  
 8   duration            250 non-null    int64  
 9   description         250 non-null    object 
 10  important_features  250 non-null    object 
dtypes: float64(1), int64(3), object(7)
memory usage: 21.6+ KB
